In [29]:
import whisper
from glob import glob
import os
import subprocess
import shutil
from os.path import join, splitext

def obtener_direcciones_archivos_carpeta(carpeta):
    
    archivos = os.listdir(carpeta)

    direcciones = [os.path.join(carpeta, archivo) for archivo in archivos]

    return direcciones



def obtener_duracion_audio(archivo_audio):
    
    
    resultado = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', archivo_audio], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    duracion = float(resultado.stdout)

    return duracion



def crear_carpeta(destino):
    if not os.path.exists(destino):
        os.makedirs(destino)

def eliminar_archivos_excepto_max_parte(archivos):
    max_parte_count = max([len(archivo.split("parte")) - 1 for archivo in archivos])

    
    [os.remove(archivo) for archivo in archivos if len(archivo.split("parte")) - 1 != max_parte_count]


def dividir_audio(archivo_audio, limite_tamanio_mb=25, primera_iteracion=True, carpeta_destino=None):
    
    tamanio_actual =  os.path.getsize(archivo_audio)

    if tamanio_actual <= limite_tamanio_mb * 1000 * 1000:
        print(f"El archivo {archivo_audio} es menor o igual a {limite_tamanio_mb} MB.")
        return [archivo_audio]

    duracion_total = obtener_duracion_audio(archivo_audio)
    
    
    duracion_mitad = duracion_total / 2.0

    if primera_iteracion:
        carpeta_destino = os.path.join(os.path.dirname(archivo_audio), "archivos_divididos")
        crear_carpeta(carpeta_destino)

    nombre_base = os.path.splitext(os.path.basename(archivo_audio))[0]

    nombre_parte1 = os.path.join(carpeta_destino, f"{nombre_base}_parte1{os.path.splitext(archivo_audio)[1]}")
    nombre_parte2 = os.path.join(carpeta_destino, f"{nombre_base}_parte2{os.path.splitext(archivo_audio)[1]}")

    comando = f'ffmpeg -i "{archivo_audio}" -t {duracion_mitad} -c copy "{nombre_parte1}" -ss {duracion_mitad} -c copy "{nombre_parte2}"'

    subprocess.run(comando, shell=True)

    partes_divididas = []
    for nombre_parte in [nombre_parte1, nombre_parte2]:
        tamanio_parte =  os.path.getsize(nombre_parte)
        if tamanio_parte > limite_tamanio_mb * 1000 * 1000:

            partes_divididas.extend(dividir_audio(nombre_parte, limite_tamanio_mb, False, carpeta_destino))
        else:
            partes_divididas.append(nombre_parte)

    
    

    return obtener_direcciones_archivos_carpeta(carpeta_destino),carpeta_destino



def carpeta(entrada,modelo):
    extensionesPermitidas = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
    model = whisper.load_model(modelo)
    

    archivosAudio = []
    for extension in extensionesPermitidas:
        archivosAudio.extend(glob(join(entrada, f'*.{extension}')))
        
    texto=""
    for archivoAudio in archivosAudio:
        
        texto+=model.transcribe(archivoAudio)["text"]
        os.remove(archivoAudio)
    
    return texto

def transforma(entrada, salida,modelo="medium"):
    
    maximo=25000000
    
    if not os.path.exists(salida):
        os.makedirs(salida)

    
    extensionesPermitidas = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
    
    

    archivosAudio = []
    for extension in extensionesPermitidas:
        archivosAudio.extend(glob(join(entrada, f'*.{extension}')))

    
    for archivoAudio in archivosAudio:
        try:
            tamaño=os.path.getsize(archivoAudio)
            nombreArchivoTxt = splitext(os.path.basename(archivoAudio))[0] + '.txt'
            rutaArchivoTxt = join(salida, nombreArchivoTxt)
            texto=""
            
            if tamaño<= maximo:
                model = whisper.load_model(modelo)
                texto=model.transcribe(archivoAudio)["text"]
            else:
                
                resultado=dividir_audio(archivoAudio)
                eliminar_archivos_excepto_max_parte(resultado[0])
                texto=carpeta(resultado[1],modelo)
                os.rmdir(resultado[1])
                
                
                
            

            
            with open(rutaArchivoTxt, 'w',encoding="utf-8") as archivoSalida:
                archivoSalida.write(texto)

            print(f"Archivo transcrito y guardado en: {rutaArchivoTxt}")

        except:
            print(texto)









In [30]:
#A="ruta"
# B="otraruta"
# Modelos ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large']
#transforma(A, B)
transforma("F:\entrevista1", "F:\entrevista1")


A
A
A
A
A
A
A
Archivo transcrito y guardado en: F:\entrevista1\220311_001.txt
